In [11]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import evaluate
import gensim
import transformers
import nltk
import torch_geometric
import json
from xml.dom import minidom
from tqdm import tqdm

from modeling import testModel

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [4]:
from os import listdir
from os.path import isfile, join

mypath = "pants_straight_sides_1000"

onlyfiles = [join(mypath,f, f + "_pattern.svg") for f in listdir(mypath) if not isfile(join(mypath, f))]


In [5]:
def read_svg_string(svg_strings):
    edge_list = []
    for i in svg_strings:
        tmp = i.split()
        prev_x = float(tmp[1])
        prev_y = float(tmp[2])
        j = 3
        while j < len(tmp):
            edge = [prev_x, prev_y]
            if tmp[j] == 'z':
                break

            if tmp[j] == 'L':
                edge.append(float(tmp[j+1]))
                edge.append(float(tmp[j+2]))
                prev_x = float(tmp[j+1])
                prev_y = float(tmp[j+2])
                edge.append((edge[0] + edge[2]) / 2)
                edge.append((edge[1] + edge[3]) / 2)
                j = j + 3
                edge_list.append(edge)
                continue
            if tmp[j] == 'Q':
                prev_x = float(tmp[j+3])
                prev_y = float(tmp[j+4])
                edge.append(float(tmp[j+3]))
                edge.append(float(tmp[j+4]))
                edge.append(float(tmp[j+1]))
                edge.append(float(tmp[j+2]))
                j = j + 5
                edge_list.append(edge)
                continue
    return edge_list
            


In [6]:
edge_index = []
for i in range(20):
    base = i // 5 * 5
    edge_index.append([i, base+ (i + 1)%5])

In [7]:
def get_data_from_spec(file_path):
    doc = minidom.parse(file_path)  # parseString also exists
    path_strings = [path.getAttribute('d') for path
                in doc.getElementsByTagName('path')]
    return read_svg_string(path_strings)

In [8]:
edge_index = torch.tensor(edge_index).to(device).permute(1,0)
edge_index.size()

torch.Size([2, 20])

In [9]:
all_data = [get_data_from_spec(i) for i in onlyfiles]
train_data = all_data[:900]
test_data = all_data[900:]

In [26]:
lr = 1e-3
num_epochs = 50

model = testModel(6).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [27]:
#training

model.train()

for epoch in tqdm(range(num_epochs), leave=False, position=0):
    running_loss = 0
    epoch_loss = 0
        
    for data in tqdm(train_data, leave=False, position=1):
        
        input_data = torch.tensor(data).to(torch.float).to(device)
            
        # Forward pass
        out = model(input_data, edge_index)
        #print(decoder_outputs.size())
            
        

        # TODO: Compute loss
        loss = criterion(out, input_data)
        epoch_loss += loss.item()
            
            
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
            
        running_loss += loss.item()
        

    print(f'Epoch {epoch + 1} | Train Loss: {(epoch_loss / len(train_data)):.4f}')
    #eval
    eval_loss = 0
    for data in  test_data:
        input_data = torch.tensor(data).to(device)
            
        # Forward pass
        out = model(input_data, edge_index)
        #print(decoder_outputs.size())
            
        

        # TODO: Compute loss
        loss = criterion(out, input_data)
        eval_loss += loss.item()
    print(f'Epoch {epoch + 1} | Eval Loss: {(eval_loss / len(test_data)):.4f}')
        
        

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1 | Train Loss: 55154.4563


  2%|▏         | 1/50 [00:08<07:16,  8.90s/it]

Epoch 1 | Eval Loss: 18178.8037


Epoch 2 | Train Loss: 6955.4244


...

Epoch 49 | Train Loss: 40.7812


 98%|█████████▊| 49/50 [08:15<00:10, 10.45s/it]

Epoch 49 | Eval Loss: 41.3324


Epoch 50 | Train Loss: 41.0715


Epoch 50 | Eval Loss: 40.6054


In [14]:
a = get_data_from_spec(onlyfiles[-1])
a= torch.tensor(a).to(device)


out = model(a, edge_index)
print(a)
print(out)

tensor([[ 60.0000, 252.0000, 183.0000, 252.0000, 121.5000, 252.0000],
        [183.0000, 252.0000, 165.0000,  60.0000, 186.0000,  77.4000],
        [165.0000,  60.0000,  90.0000,  60.0000, 127.5000,  60.0000],
        [ 90.0000,  60.0000,  60.0000, 150.0000, 103.5000, 139.5000],
        [ 60.0000, 150.0000,  60.0000, 252.0000,  70.2000, 155.1000],
        [243.0000, 252.0000, 336.0000, 252.0000, 289.5000, 252.0000],
        [336.0000, 252.0000, 336.0000, 150.0000, 325.8000, 155.1000],
        [336.0000, 150.0000, 306.0000,  60.0000, 297.0000, 138.0000],
        [306.0000,  60.0000, 261.0000,  60.0000, 283.5000,  60.0000],
        [261.0000,  60.0000, 243.0000, 252.0000, 240.0000,  77.4000],
        [396.0000, 252.0000, 519.0000, 252.0000, 457.5000, 252.0000],
        [519.0000, 252.0000, 519.0000, 150.0000, 508.8000, 155.1000],
        [519.0000, 150.0000, 489.0000,  60.0000, 475.5000, 139.5000],
        [489.0000,  60.0000, 414.0000,  60.0000, 451.5000,  60.0000],
        [414.0000,  

In [18]:
def convert_to_4(data):
    start_p = np.array([data[0], data[1]])
    end_p = np.array([data[2], data[3]])
    center_p = np.array([data[4], data[5]])

    line = end_p - start_p
    p_line = np.empty_like(line)
    p_line[0] = - line[1]
    p_line[1] = line[0]

    v = center_p - start_p
    cx = np.dot(line, v) / np.linalg.norm(line)
    cy = np.dot(p_line, v) / np.linalg.norm(p_line)

    return [line[0], line[1], cx, cy]

[0.0, -297.0, 267.3, 29.699999999999985]